Reference: https://geodacenter.github.io/workbook/6a_local_auto/lab6a.html#local-geary

# Univariate local geary

Interestingly, following [this equation](https://www.biomedware.com/files/documentation/spacestat/Statistics/Gearys_C/Geary_s_C_statistic.htm) which explicitly calls for standardization of input data. We also do NOT divide by 2.

$$ c_i = \sum_j w_{ij} (z_i - z_j)^2 $$ 

where: 

$z_i = x_i - \bar{x}$ and $z_j = x_j - \bar{x}$, and $w_{ij}$ are the elements of the row-standardized binary symmetric spatial weight matrix W. 

or, $$ c_i = (1/m^2) * \sum_j w_{ij} (x_i - x_j)^2 $$

where,

$$ m^2 = \sum_i (x_i−\bar{x})^2/n $$

## Load in example data

In [75]:
import libpysal as lp
import geopandas as gpd
from scipy import stats
guerry = lp.examples.load_example('Guerry')
guerry_ds = gpd.read_file(guerry.get_path('Guerry.shp'))

In [76]:
wq = lp.weights.Queen.from_dataframe(guerry_ds)

In [77]:
wq[0]

{66: 1.0, 35: 1.0, 68: 1.0, 36: 1.0}

In [78]:
wq.transform = 'r'
wq[0]

{66: 0.25, 35: 0.25, 68: 0.25, 36: 0.25}

In [79]:
x = guerry_ds['Donatns']

print("x_i is", x[0])
print("x_j are", x[66], x[35], x[68], x[36])

x_i is 5098
x_j are 1983 4077 3710 3012


In [80]:
# Calculate score
zscore_x = (x - np.mean(x))/np.std(x)
zscore_x

0    -0.336188
1     0.450441
2     0.879023
3    -0.825375
4     0.049370
        ...   
80    1.512380
81    0.454785
82    1.467288
83   -0.555029
84   -0.506214
Name: Donatns, Length: 85, dtype: float64

# Build expected local geary values

In [81]:
adj_list = wq.to_adjlist(remove_symmetric=False)
adj_list.head()

,focal,neighbor,weight
0,0,66,0.250000
1,0,35,0.250000
2,0,68,0.250000
3,0,36,0.250000
4,1,48,0.166667


In [82]:
import pandas as pd
zseries = pd.Series(zscore_x, index=wq.id_order)
zseries[0:5]

0   -0.336188
1    0.450441
2    0.879023
3   -0.825375
4    0.049370
Name: Donatns, dtype: float64

In [83]:
# Define z_i
zi = zseries.loc[adj_list.focal].values
zi[0:5]

array([-0.33618783, -0.33618783, -0.33618783, -0.33618783,  0.45044136])

In [84]:
# Define zj
zj = zseries.loc[adj_list.neighbor].values
zj[0:5]

array([-0.98050808, -0.54737594, -0.62328783, -0.76766521, -0.5709562 ])

In [96]:
(zi-zj)[0:5]

array([0.64432025, 0.21118812, 0.2871    , 0.43147738, 1.02139756])

In [85]:
(zi-zj)**2

array([4.15148590e-01, 4.46004201e-02, 8.24264123e-02, 1.86172733e-01,
       1.04325299e+00, 2.67617709e-01, 5.53872475e-01, 4.94589963e-01,
       3.37591065e-01, 6.63159957e-01, 8.53015374e-02, 2.25693761e+00,
       2.46439290e-05, 1.16134943e-02, 2.42399264e+00, 1.07389771e+00,
       7.65178635e-01, 9.46039788e-02, 3.45083461e-03, 3.94302864e-04,
       3.56105843e-01, 7.65178635e-01, 7.30833213e-01, 3.38135670e-02,
       5.51412189e-03, 2.84791404e-03, 8.35857038e-03, 8.38578139e-04,
       5.63859940e-02, 1.61356296e-01, 2.54096407e-01, 2.67617709e-01,
       2.07831152e-01, 3.94302864e-02, 6.74942450e-02, 2.80848393e+00,
       5.39192908e-03, 7.01774384e-03, 4.88011403e-02, 1.90915149e-02,
       1.22921222e-01, 3.48097208e-02, 3.94302864e-02, 3.21607124e-02,
       3.74861542e-03, 3.51346459e+00, 3.32831663e-02, 7.76287613e-02,
       1.13674401e-03, 1.68241408e-01, 2.42349082e-03, 5.86679962e-02,
       1.00285431e-01, 2.30504594e-02, 7.79749315e-04, 4.88011403e-02,
      

Multiply by spatial weights

In [86]:
sum(list(wq.weights.values()), []) * (zi-zj)**2

array([1.03787147e-01, 1.11501050e-02, 2.06066031e-02, 4.65431831e-02,
       1.73875498e-01, 4.46029515e-02, 9.23120791e-02, 8.24316606e-02,
       5.62651775e-02, 1.10526659e-01, 1.42169229e-02, 3.76156268e-01,
       4.10732150e-06, 1.93558239e-03, 4.03998774e-01, 1.78982952e-01,
       1.91294659e-01, 2.36509947e-02, 8.62708652e-04, 9.85757159e-05,
       1.18701948e-01, 2.55059545e-01, 2.43611071e-01, 4.83050957e-03,
       7.87731699e-04, 4.06844863e-04, 1.19408148e-03, 1.19796877e-04,
       8.05514201e-03, 2.30508994e-02, 8.46988022e-02, 8.92059030e-02,
       6.92770507e-02, 1.31434288e-02, 2.24980817e-02, 9.36161309e-01,
       1.07838582e-03, 1.40354877e-03, 9.76022807e-03, 3.81830297e-03,
       2.45842445e-02, 6.96194416e-03, 7.88605727e-03, 6.43214247e-03,
       7.49723084e-04, 7.02692918e-01, 4.75473805e-03, 1.10898230e-02,
       1.62392001e-04, 2.40344868e-02, 3.46212974e-04, 8.38114231e-03,
       1.43264902e-02, 7.68348646e-03, 2.59916438e-04, 1.62670468e-02,
      

In [87]:
test = sum(list(wq.weights.values()), []) * (zi-zj)**2

In [88]:
test 

array([1.03787147e-01, 1.11501050e-02, 2.06066031e-02, 4.65431831e-02,
       1.73875498e-01, 4.46029515e-02, 9.23120791e-02, 8.24316606e-02,
       5.62651775e-02, 1.10526659e-01, 1.42169229e-02, 3.76156268e-01,
       4.10732150e-06, 1.93558239e-03, 4.03998774e-01, 1.78982952e-01,
       1.91294659e-01, 2.36509947e-02, 8.62708652e-04, 9.85757159e-05,
       1.18701948e-01, 2.55059545e-01, 2.43611071e-01, 4.83050957e-03,
       7.87731699e-04, 4.06844863e-04, 1.19408148e-03, 1.19796877e-04,
       8.05514201e-03, 2.30508994e-02, 8.46988022e-02, 8.92059030e-02,
       6.92770507e-02, 1.31434288e-02, 2.24980817e-02, 9.36161309e-01,
       1.07838582e-03, 1.40354877e-03, 9.76022807e-03, 3.81830297e-03,
       2.45842445e-02, 6.96194416e-03, 7.88605727e-03, 6.43214247e-03,
       7.49723084e-04, 7.02692918e-01, 4.75473805e-03, 1.10898230e-02,
       1.62392001e-04, 2.40344868e-02, 3.46212974e-04, 8.38114231e-03,
       1.43264902e-02, 7.68348646e-03, 2.59916438e-04, 1.62670468e-02,
      

In [ ]:
# Create a df that uses the adjacency list focal values and the BBs counts
temp = pd.DataFrame(adj_list.focal.values, test).reset_index()
temp

In [ ]:
# Temporarily rename the columns
temp.columns = ['Eij', 'ID']
temp = temp.groupby(by='ID').sum()

In [ ]:
temp.Eij.values[0:5]

## Start working on inference

# Start building function

In [27]:
import numpy as np
import pandas as pd
import warnings
from scipy import sparse
from scipy import stats
from sklearn.base import BaseEstimator
import libpysal as lp


class Local_Geary(BaseEstimator):
    """Local Geary - Univariate"""

    def __init__(self, connectivity=None, inference=None):
        """
        Initialize a Local_Geary estimator

        Arguments
        ---------
        connectivity     : scipy.sparse matrix object
                           the connectivity structure describing the
                           relationships between observed units.
        inference        : str
                           describes type of inference to be used. options are
                           "chi-square" or "permutation" methods.

        Attributes
        ----------
        localG           : numpy array
                           Array of Local Geary values for each spatial unit.
        pval             : numpy array
                           P-values for inference based on either
                           "chi-square" or "permutation" methods.
        """

        self.connectivity = connectivity
        self.inference = inference

    def fit(self, x):
        """
        Arguments
        ---------
        x                : numpy.ndarray
                           array containing continuous data

        Returns
        -------
        the fitted estimator.

        Notes
        -----
        Technical details and derivations can be found in :cite:`Anselin1995`.

        Examples
        --------
        Guerry data replication GeoDa tutorial
        >>> import libpysal
        >>> import geopandas as gpd
        >>> guerry = lp.examples.load_example('Guerry')
        >>> guerry_ds = gpd.read_file(guerry.get_path('Guerry.shp'))
        >>> w = libpysal.weights.Queen.from_dataframe(guerry_ds)
        """
        x = np.asarray(x).flatten()

        w = self.connectivity
        w.transform = 'r'

        self.localG = self._statistic(x, w)

        if self.inference is None:
            return self
        #elif self.inference == 'chi-square':
        #    if a != 2:
        #        warnings.warn(f'Chi-square inference assumes that a=2, but \
        #        a={a}. This means the inference will be invalid!')
        #    else:
        #        dof = 2/self.VarHi
        #        Zi = (2*self.Hi)/self.VarHi
        #        self.pval = 1 - stats.chi2.cdf(Zi, dof)
        else:
            raise NotImplementedError(f'The requested inference method \
            ({self.inference}) is not currently supported!')

        return self

    @staticmethod
    def _statistic(x, w):
        # Caclulate z-scores for x
        zscore_x = (x - np.mean(x))/np.std(x)
        # Create focal (xi) and neighbor (zi) values
        adj_list = w.to_adjlist(remove_symmetric=False)
        zseries = pd.Series(zscore_x, index=wq.id_order)
        zi = zseries.loc[adj_list.focal].values
        zj = zseries.loc[adj_list.neighbor].values
        # Carry out local Geary calculation
        gs = sum(list(wq.weights.values()), []) * (zi-zj)**2
        # Reorganize data
        adj_list_gs = pd.DataFrame(adj_list.focal.values, gs).reset_index()
        adj_list_gs.columns = ['gs', 'ID']
        adj_list_gs = adj_list_gs.groupby(by='ID').sum()
        
        localG = adj_list_gs.gs.values
        
        return (localG)

In [28]:
functest = Local_Geary(connectivity=wq).fit(x)
functest.localG

array([1.82087039e-01, 5.60014026e-01, 9.75294606e-01, 2.15906938e-01,
       6.17372564e-01, 3.84450059e-02, 2.43181756e-01, 9.71802819e-01,
       4.06447101e-02, 7.24722785e-01, 6.30952854e-02, 2.42104497e-02,
       1.59496916e+01, 9.29326006e-01, 9.65188634e-01, 1.32383286e+00,
       3.31775497e-01, 2.99446505e+00, 9.43946814e-01, 2.99570159e+00,
       3.66702291e-01, 2.09592365e+00, 1.46515861e+00, 1.82118455e-01,
       3.10216680e+00, 5.43063937e-01, 5.74532559e+00, 4.79160197e-02,
       1.58993089e-01, 7.18327253e-01, 1.24297849e+00, 8.72629331e-02,
       7.52809650e-01, 4.56515485e-01, 3.86766562e-01, 1.17632604e-01,
       6.90884685e-01, 2.87206102e+00, 4.10455112e-01, 4.04349959e-01,
       1.14211758e-01, 9.59519953e-01, 3.51347976e-01, 7.30240974e-01,
       4.40370938e-01, 7.20360356e-02, 1.66241706e+00, 5.83258909e+00,
       2.30332507e-01, 4.38369688e-01, 8.41461470e-01, 1.52959486e+00,
       4.32157479e-02, 2.08325903e+00, 1.19722984e+00, 1.28169257e+00,
      

# Multivariate Local Geary

$$ c_i = \sum_{h=1}^m \sum_j w_{ij} (x_{hi} - x_{hj})^2 $$

Load in the sample data

In [14]:
import libpysal as lp
import geopandas as gpd
from scipy import stats
import pandas as pd
guerry = lp.examples.load_example('Guerry')
guerry_ds = gpd.read_file(guerry.get_path('Guerry.shp'))

In [5]:
wq = lp.weights.Queen.from_dataframe(guerry_ds)

In [9]:
x = guerry_ds['Donatns']
y = guerry_ds['Suicids']

In [18]:
variables = [x,y]
variables

[0      5098
 1      8901
 2     10973
 3      2733
 4      6962
       ...  
 80    14035
 81     8922
 82    13817
 83     4040
 84     4276
 Name: Donatns, Length: 85, dtype: int64,
 0      35039
 1      12831
 2     114121
 3      14238
 4      16171
        ...  
 80     67963
 81     21851
 82     33497
 83     33029
 84     12789
 Name: Suicids, Length: 85, dtype: int64]

Standardize each variable

In [105]:
from scipy import stats
zseries = [stats.zscore(i) for i in variables]

Build the adj lists

In [106]:
adj_list = wq.to_adjlist(remove_symmetric=False)

In [109]:
# The zseries
zseries = [pd.Series(i, index=wq.id_order) for i in zseries]

In [110]:
zseries

[0    -0.336188
 1     0.450441
 2     0.879023
 3    -0.825375
 4     0.049370
         ...   
 80    1.512380
 81    0.454785
 82    1.467288
 83   -0.555029
 84   -0.506214
 Length: 85, dtype: float64,
 0    -0.047195
 1    -0.756433
 2     2.478379
 3    -0.711499
 4    -0.649766
         ...   
 80    1.004270
 81   -0.468369
 82   -0.096441
 83   -0.111387
 84   -0.757774
 Length: 85, dtype: float64]

In [111]:
# The focal values
focal = [zseries[i].loc[adj_list.focal].values for
         i in range(len(variables))]
# The neighbor values
neighbor = [zseries[i].loc[adj_list.neighbor].values for
            i in range(len(variables))]

In [116]:
gs = sum(list(wq.weights.values()), []) * (np.array(focal) - np.array(neighbor))**2

In [124]:
temp = pd.DataFrame(gs).T

In [125]:
temp['ID'] = adj_list.focal.values

In [131]:
adj_list_gs = temp.groupby(by='ID').sum()
adj_list_gs.head()

,0,1
ID,,
0,0.182087,0.125550
1,0.560014,0.047105
2,0.975295,4.934146
3,0.215907,0.030372
4,0.617373,0.158547


In [132]:
k = len(variables)
k

2

In [133]:
adj_list_gs.sum(axis=1)/k

ID
0     0.153819
1     0.303560
2     2.954720
3     0.123140
4     0.387960
        ...   
80    1.657430
81    0.525764
82    0.645337
83    0.717948
84    0.216181
Length: 85, dtype: float64

In [135]:
import numpy as np
import pandas as pd
import warnings
from scipy import sparse
from scipy import stats
from sklearn.base import BaseEstimator
import libpysal as lp


class Local_Geary_MV(BaseEstimator):
    """Local Geary - Univariate"""

    def __init__(self, connectivity=None, inference=None):
        """
        Initialize a Local_Geary estimator

        Arguments
        ---------
        connectivity     : scipy.sparse matrix object
                           the connectivity structure describing the
                           relationships between observed units.
        inference        : str
                           describes type of inference to be used. options are
                           "chi-square" or "permutation" methods.

        Attributes
        ----------
        localG           : numpy array
                           Array of Local Geary values for each spatial unit.
        pval             : numpy array
                           P-values for inference based on either
                           "chi-square" or "permutation" methods.
        """

        self.connectivity = connectivity
        self.inference = inference

    def fit(self, variables):
        """
        Arguments
        ---------
        variables        : numpy.ndarray
                           array containing continuous data

        Returns
        -------
        the fitted estimator.

        Notes
        -----
        Technical details and derivations can be found in :cite:`Anselin1995`.

        Examples
        --------
        Guerry data replication GeoDa tutorial
        >>> import libpysal
        >>> import geopandas as gpd
        >>> guerry = lp.examples.load_example('Guerry')
        >>> guerry_ds = gpd.read_file(guerry.get_path('Guerry.shp'))
        >>> w = libpysal.weights.Queen.from_dataframe(guerry_ds)
        """
        self.variables = np.array(variables, dtype='float')

        w = self.connectivity
        w.transform = 'r'

        self.localG = self._statistic(variables, w)

        if self.inference is None:
            return self
        #elif self.inference == 'chi-square':
        #    if a != 2:
        #        warnings.warn(f'Chi-square inference assumes that a=2, but \
        #        a={a}. This means the inference will be invalid!')
        #    else:
        #        dof = 2/self.VarHi
        #        Zi = (2*self.Hi)/self.VarHi
        #        self.pval = 1 - stats.chi2.cdf(Zi, dof)
        else:
            raise NotImplementedError(f'The requested inference method \
            ({self.inference}) is not currently supported!')

        return self

    @staticmethod
    def _statistic(variables, w):
        # Caclulate z-scores for input variables
        zseries = [stats.zscore(i) for i in variables]
        # Define denominator adjustment
        k = len(variables)
        # Create focal and neighbor values
        zseries = [pd.Series(i, index=wq.id_order) for i in zseries]
        focal = [zseries[i].loc[adj_list.focal].values for
                 i in range(len(variables))]
        neighbor = [zseries[i].loc[adj_list.neighbor].values for
                    i in range(len(variables))]
        # Carry out local Geary calculation
        gs = sum(list(wq.weights.values()), []) * \
        (np.array(focal) - np.array(neighbor))**2
        # Reorganize data
        temp = pd.DataFrame(gs).T
        temp['ID'] = adj_list.focal.values
        adj_list_gs = temp.groupby(by='ID').sum()
        localG = adj_list_gs.sum(axis=1)/k
        
        return (localG)

In [137]:
functest = Local_Geary_MV(connectivity=wq).fit([x,y])
functest.localG

ID
0     0.153819
1     0.303560
2     2.954720
3     0.123140
4     0.387960
        ...   
80    1.657430
81    0.525764
82    0.645337
83    0.717948
84    0.216181
Length: 85, dtype: float64